## paper: [Contrastive Decoding: Open-ended Text Generation as Optimization](https://arxiv.org/abs/2210.15097)


In [ ]:
import torch
import transformers

tokenizer = transformers.AutoTokenizer.from_pretrained('gpt2')
amateur_lm = transformers.AutoModelForCausalLM.from_pretrained('gpt2')
expert_lm = transformers.AutoModelForCausalLM.from_pretrained('gpt2-medium')

encoding = tokenizer("Barack Obama was born in Honolulu, Hawaii. He was born in", return_tensors='pt')
logits = expert_lm(**encoding).logits